# Penney's game sim

functions
- generate 52-bit string function
- simulate game functoin

loop through 8x8 variations and simulate. append data to a dictionary.
- number of points p1 scored in this variation
- number of points p2 scored in this variation
- number of games played total in this variation

take data and make a heatmap. each square is the number of points p1 has scored in that variation **divided by** the number of times this variation has been played

make a bunch of heatmaps. turn it into a gif. delete all the heatmaps

In [1]:
import random
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import glob
import imageio
import os

# generate half ones and half zeros for 52-bit number
def generate_half_ones_half_zeros():
    bits = ['1'] * 26 + ['0'] * 26
    random.shuffle(bits)
    return int(''.join(bits), 2)

# simulate the game given a deck and two patterns to look for
def pattern_game_52bits(n, player1_pattern, player2_pattern):
    binary_str = bin(n)[2:].zfill(52)
    
    player1_points = 0
    player2_points = 0
    
    bits_traversed = 0
    last_pattern_position = 0
    
    i = 0
    while i <= len(binary_str) - 3:
        current_bits = binary_str[i:i+3]
        bits_traversed += 1
        
        if current_bits == player1_pattern:
            # add 2 because we are looking at 3 bits at a time
            player1_points += bits_traversed - last_pattern_position + 2
            last_pattern_position = bits_traversed
            i += 2
        elif current_bits == player2_pattern:
            player2_points += bits_traversed - last_pattern_position + 2
            last_pattern_position = bits_traversed
            i += 2
        i += 1
    
    return player1_points, player2_points


# generate all possible 3-bit combinations
three_bit_combos = [''.join(p) for p in product('01', repeat=3)]

# dict to store the sum of scores and count of games for each pattern combination
scores = {(p1, p2): {'player1_sum': 0, 'player2_sum': 0, 'count': 0} for p1 in three_bit_combos for p2 in three_bit_combos}

# gif making
filenames = []
gif_filename = 'heatmap_animation.gif'

# iterations
num_iterations = 500
save_interval = 20


for iteration in range(1, num_iterations + 1):
    #generate deck
    random_52_bit_integer = generate_half_ones_half_zeros()
    for player1_pattern in three_bit_combos:
        for player2_pattern in three_bit_combos:
            # simulate the game
            player1_points, player2_points = pattern_game_52bits(random_52_bit_integer, player1_pattern, player2_pattern)

            # use the variation as a key in saving the game data
            key = (player1_pattern, player2_pattern)
            
            # update the sum and count for each pattern combination
            scores[key]['player1_sum'] += player1_points
            scores[key]['player2_sum'] += player2_points
            scores[key]['count'] += 1

    # after trying each combo pattern for this deck, we need to add this data to our previous data
    if iteration % save_interval == 0:
        # create dataframe with the average points for each combination
        results = [{'Player 1 Pattern': k[0],
                    'Player 2 Pattern': k[1],
                    'Player 1 Points': scores[k]['player1_sum'] / scores[k]['count'],
                    'Player 2 Points': scores[k]['player2_sum'] / scores[k]['count']}
                   for k in scores]

        df = pd.DataFrame(results)

        # pivot table for heatmap
        heatmap_data = df.pivot(index="Player 1 Pattern", columns="Player 2 Pattern", values="Player 1 Points")

        # create heatmap of average score
        plt.figure(figsize=(10, 8))
        sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="YlGnBu")
        plt.title(f"Heatmap of Player 1 Average Points after {iteration} iterations")
        plt.xlabel("Player 2 Pattern")
        plt.ylabel("Player 1 Pattern")
        
        # save heatmap image
        filename = f"heatmap_{iteration}.png"
        plt.savefig(filename)
        filenames.append(filename)
        plt.close()

# create the gif
with imageio.get_writer(gif_filename, mode='I', duration=0.5) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# now that we have the gif, remove all the images from our files
for filename in filenames:
    os.remove(filename)

print(f"GIF saved as {gif_filename}")


/tmp/ipykernel_922/860011405.py:107: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved as heatmap_animation.gif
